In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.preprocessing import label_binarize

In [5]:
from sklearn.preprocessing import LabelBinarizer

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
datapath = r'C:\Sports-Type-Classifier\dataset\data'
outputmodel = r'C:\Sports-Type-Classifier\video_classification_model\videoclassificationModel'
ouputlabelbinarize = r'C:\Sports-Type-Classifier\video_classification_model\videoclassificationbinarizer'
epoch= 25

In [9]:
Sports_Labels = set(['boxing', 'swimming', 'table_tennis'])
print('Images is being loaded..')
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)

Images is being loaded..


In [10]:
data = np.array(data)
labels = np.array(labels)
#hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [11]:
(X_train, X_test, y_train, y_test)= train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

In [13]:
traininaugmentation = ImageDataGenerator(
rotation_range=30,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode='nearest')
validationaugmentation= ImageDataGenerator()
mean = np.array([123.68, 116.77, 103.939], dtype='float32')
traininaugmentation.mean = mean
validationaugmentation.mean = mean

In [26]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
#from keras.applications import ResNet50
#from keras.applications.resnet50 import ResNet50
from keras.layers.core import Flatten
from keras.layers.core import Dense
from tensorflow.keras.applications.resnet import ResNet50

from keras.layers.core import Dropout
from keras.models import Model

In [27]:
baseModel = ResNet50(weights='imagenet', include_top= False, input_tensor = Input(shape=(224,224,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation='softmax')(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodellayers in baseModel.layers:
    basemodellayers.trainable = False

ValueError: Unexpectedly found an instance of type `<class 'keras.engine.keras_tensor.KerasTensor'>`. Expected a symbolic tensor instance.

In [23]:
from keras.optimizers import SGD

In [50]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [51]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [53]:
History= model.fit_generator(
traininaugmentation.flow(X_train,y_train, batch_size=32),
steps_per_epoch=len(X_train) // 32,
validation_data= validationaugmentation.flow(X_test, y_test),
validation_steps=len(X_test) //32,
epochs= epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
52/52 [==============================] - 78s 1s/step - loss: 1.0565 - accuracy: 0.5551 - val_loss: 0.4330 - val_accuracy: 0.8245
Epoch 2/25
52/52 [==============================] - 77s 1s/step - loss: 0.6456 - accuracy: 0.7361 - val_loss: 0.2862 - val_accuracy: 0.8990
Epoch 3/25
52/52 [==============================] - 78s 1s/step - loss: 0.4921 - accuracy: 0.7952 - val_loss: 0.2298 - val_accuracy: 0.9255
Epoch 4/25
52/52 [==============================] - 78s 1s/step - loss: 0.4005 - accuracy: 0.8422 - val_loss: 0.2010 - val_accuracy: 0.9255
Epoch 5/25
52/52 [==============================] - 79s 2s/step - loss: 0.3831 - accuracy: 0.8513 - val_loss: 0.1807 - val_accuracy: 0.9399
Epoch 6/25
52/52 [==============================] - 84s 2s/step - loss: 0.3399 - accuracy: 0.8653 - val_loss: 0.1656 - val_accuracy: 0.9423
Epoch 7/25
52/52 [==============================] - 79s 2s/step - loss: 0.3078 - acc

In [54]:
import pickle

In [55]:
model.save(outputmodel)
lbinarizer = open(r'C:\Sports-Type-Classifier\video_classification_model\videoclassificationbinarizer.pickle', 'wb')
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\KHJ\Sports-Type-Classifier\video_classification_model\videoclassificationModel\assets
